In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [3]:
ingredient1 = pd.read_csv("/Users/hyeseon/Downloads/ingredients v1.csv")

In [6]:
ingredient1.head()

,id,asins,brand,categories,dateAdded,dateUpdated,ean,features.key,features.value,manufacturer,manufacturerNumber,name,sizes,upc,weight,Unnamed: 15
0,AVphBRHOilAPnD_x0OrE,B00HXST15C,Simon Fischer,"Grocery & Gourmet Food,Food,Grocery",2017-01-07T20:13:17Z,2017-06-30T16:48:02Z,41642026706,Ingredients,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin.",Sokol And Company,33829,Simon Fischer Fruit Bttr Prune Lekvar,NaN,41642026706,10.6 pounds,NaN
1,AVpfNFy1LJeJML434ma2,"B008VT0W8C,B0092F8OJ8",McCormick,"Grocery & Gourmet Food,Food,Grocery",2016-02-05T21:08:01Z,2017-04-12T13:54:04Z,52100018164,Ingredients,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,...","McCormick & Co, Inc",MCLANE500373852,McCORMICK GRILL MATES MOLASSES BACON SEASONING...,NaN,52100018164,3.5 ounces,NaN
2,AVpgT49VLJeJML43MJEz,B00CHTWZ2S,Jolly Time,"Grocery & Gourmet Food,Grocery",2014-02-18T01:51:23Z,2017-08-07T16:32:11Z,70670004141,Ingredients,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ...",Reese's,NaN,Jolly Time Popcorn,NaN,70670004141,1.8 pounds,NaN
3,AVphYgnzLJeJML43aPp2,B002JJYNVW,Ziyad,"Grocery & Gourmet Food,grocery",2014-11-04T11:39:27Z,2017-09-04T05:10:37Z,NaN,Ingredients,Mechanically hulled seasame seeds.Allergy Info...,Ziyad,NaN,Ziyad Tahini Sesame Sauce,NaN,74265001560,1.6 pounds,NaN
4,AVpiS0bOLJeJML43kRsh,B00290W1CY,Fla-Vor-Ice,"Grocery & Gourmet Food,grocery",2014-02-18T02:32:12Z,2017-08-26T01:45:28Z,"723,929,530,040,072,000,000,000",Ingredients,FALSE,Fla-Vor-Ice,NaN,Fla-Vor-Ice Plus Giant Pops,NaN,"72,392,952,335,072,300,000,000",18 pounds,NaN


In [7]:
ingredient1.shape

(10000, 16)

In [9]:
sum(ingredient1.duplicated()) #no duplicates

0

In [15]:
ingredient1 = ingredient1[['name','features.value']].copy()

In [16]:
ingredient1.isna().sum()

name               1
features.value    28
dtype: int64

In [20]:
ingredient1.dropna(inplace=True)
ingredient1.shape

(9971, 2)

In [21]:
ingredient1.head()

,name,features.value
0,Simon Fischer Fruit Bttr Prune Lekvar,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin."
1,McCORMICK GRILL MATES MOLASSES BACON SEASONING...,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,..."
2,Jolly Time Popcorn,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ..."
3,Ziyad Tahini Sesame Sauce,Mechanically hulled seasame seeds.Allergy Info...
4,Fla-Vor-Ice Plus Giant Pops,FALSE


In [22]:
ingredient2 = pd.read_csv("/Users/hyeseon/Downloads/food_ingredients_and_allergens.csv")

In [23]:
ingredient2.head()

,Food Product,Main Ingredient,Sweetener,Fat/Oil,Seasoning,Allergens,Prediction
0,Almond Cookies,Almonds,Sugar,Butter,Flour,"Almonds, Wheat, Dairy",Contains
1,Almond Cookies,Almonds,Sugar,Butter,Flour,"Almonds, Wheat, Dairy",Contains
2,Chicken Noodle Soup,Chicken broth,NaN,NaN,Salt,"Chicken, Wheat, Celery",Contains
3,Chicken Noodle Soup,Chicken broth,NaN,NaN,Salt,"Chicken, Wheat, Celery",Contains
4,Cheddar Cheese,Cheese,NaN,NaN,Salt,Dairy,Contains


In [24]:
ingredient2.shape

(399, 7)

In [47]:
ingredient2['Prediction'].unique()

array(['Contains', 'Does not contain', nan], dtype=object)

In [63]:
ingredient2['Prediction'].value_counts()

Prediction
Contains            255
Does not contain    143
Name: count, dtype: int64

In [66]:
ingredient2.dropna(inplace=True)

In [67]:
ingredient2.isnull().sum()

Food Product       0
Main Ingredient    0
Sweetener          0
Fat/Oil            0
Seasoning          0
Allergens          0
Prediction         0
dtype: int64

In [174]:
def categorize_allergens(allergens):
    categories = {
        'Cereals containing gluten': ['wheat', 'spelt', 'barley', 'oat'],
        'Crustaceans and their products': ['shrimps', 'crabs', 'lobsters', 'crayfish'],
        'Eggs and egg products': ['eggs','egg white','egg yolk'],
        'Fish and fish products': ['anchovies', 'sardines', 'caviar', 'salmon', 'tuna', 'cod', 'fish sauce','fish'],
        'Peanuts and peanut products': ['peanuts', 'peanut oil', 'groundnuts', 'arachis oil'],
        'Soybeans and soy products': ['soybeans', 'soy protein', 'soy sauce', 'edamame'],
        'Milk and milk products': ['milk', 'lactose', 'casein', 'cheese', 'cream', 'yogurt', 'whey'],
        'Nuts (tree nuts) and nut products': ['almonds', 'walnuts', 'cashews', 'pecans', 'hazelnuts', 'brazilnuts','nuts'],
        'Sesame seeds and sesame products': ['sesame seeds', 'sesame oil', 'tahini'],
        'Sulphur dioxide and sulphites': [],
        'Mustard and mustard products': ['mustard seeds', 'mustard powder', 'dijon mustard'],
        'Lupin and lupin products': ['lupin beans', 'lupin flour'],
        'Mollusks and their products': ['clams', 'mussels', 'oysters'],
    }
    categorized_allergens = {category: [] for category in categories.keys()}

    for allergen in allergens:
            for category, terms in categories.items():
                if allergen in terms:
                    categorized_allergens[category].append(allergen)
                    break

    return categorized_allergens


In [175]:
categorized_allergens = categorize_allergens(allergens)


In [43]:
def categorize_allergens(text):
    text = text.lower()  # Convert text to lowercase
    detected_allergens = []
    for category, allergens in categorized_allergens.items():
        for allergen in allergens:
            if allergen in text:
                detected_allergens.append(category)
                break
    return ', '.join(detected_allergens) if detected_allergens else 'No allergens'

ingredient1['Allergens'] = ingredient1['features.value'].apply(categorize_allergens)

In [46]:
ingredient1.head()
ingredient1.tail()

,name,features.value,Allergens
9995,Wright's Liquid Smoke - 3.5 Oz,"water,natural hickory smoke flavor.",No allergens
9996,Eden Foods Organic Pocket Snacks - Tamari Almo...,"Organic Dry Roasted Almonds,Organic Tamari Soy...","Soybeans and soy products, Nuts (tree nuts) an..."
9997,"Happy Belly Fancy Whole Cashews, 44 Ounce","Cashews, Vegetable Oil (May Contain One Or Mor...",Nuts (tree nuts) and nut products
9998,QuakerÂ® PoppedÂ® Cheddar Cheese Rice Crisps 6...,"Whole Grain Brown Rice Flour,Degerminated Mill...",Milk and milk products
9999,Alpen All Natural Muesli Original,"Non-GMO Whole Grain Wheat,Non-GMO Whole Grain ...","Cereals containing gluten, Milk and milk produ..."


In [150]:
ingredient1[ingredient1['name'].str.contains('pizza', case=False)]

In [49]:
def create_prediction(Allergens):
    if Allergens == 'No allergens':
        return 'Does not contain'
    else:
        return 'Contains'

ingredient1['Prediction'] = ingredient1['Allergens'].apply(create_prediction)

In [51]:
ingredient1.head()
ingredient1.tail()

,name,features.value,Allergens,Prediction
9995,Wright's Liquid Smoke - 3.5 Oz,"water,natural hickory smoke flavor.",No allergens,Does not contain
9996,Eden Foods Organic Pocket Snacks - Tamari Almo...,"Organic Dry Roasted Almonds,Organic Tamari Soy...","Soybeans and soy products, Nuts (tree nuts) an...",Contains
9997,"Happy Belly Fancy Whole Cashews, 44 Ounce","Cashews, Vegetable Oil (May Contain One Or Mor...",Nuts (tree nuts) and nut products,Contains
9998,QuakerÂ® PoppedÂ® Cheddar Cheese Rice Crisps 6...,"Whole Grain Brown Rice Flour,Degerminated Mill...",Milk and milk products,Contains
9999,Alpen All Natural Muesli Original,"Non-GMO Whole Grain Wheat,Non-GMO Whole Grain ...","Cereals containing gluten, Milk and milk produ...",Contains


In [64]:
ingredient1['Prediction'].value_counts()

Prediction
Contains            5085
Does not contain    4886
Name: count, dtype: int64

In [ ]:
#Recommendation Model

In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
######receive input data#######

In [172]:
input_food = 'pizza'
allergen_requirements = 'milk'

In [173]:
recommendation = pd.DataFrame()
recommendation['food_data'] = ingredient1['features.value'] +  " | "  + ingredient1['Allergens']
recommendation.head()

,food_data
0,"Dried Prunes,Water,Corn Syrup,Sugar,Pectin. | ..."
1,"Salt,Sugar,Molasses (Refinery Syrup, Molasses,..."
2,"Salt, Yellow 5 Lake, Tricalcium Phosphate And ..."
3,Mechanically hulled seasame seeds.Allergy Info...
4,FALSE | No allergens


In [135]:
tfidf_vectorizer = TfidfVectorizer()

In [140]:
tfidf_matrix = tfidf_vectorizer.fit_transform(recommendation['food_data'])

In [164]:
input_food_index = ingredient1[ingredient1['name'].str.contains(input_food, case = False)].index[0]

In [165]:
cosine_similarities = linear_kernel(tfidf_matrix[input_food_index], tfidf_matrix).flatten()

In [166]:
similar_food_indices = cosine_similarities.argsort()[::-1][1:]

In [167]:
top_N = 3

In [168]:
top_similar_foods = ingredient1['name'].iloc[similar_food_indices[:top_N]]

In [169]:
print("Recommended foods similar to {input_food} based on ingredients and allergens : ")
print(top_similar_foods)

Recommended foods similar to {input_food} based on ingredients and allergens : 
5130           Lunch Combinations, Pepperoni Pizza, 1 kit
1798    Tombstone Cheese Stuffed Crust Pepperoni Pizza...
2524    Hormel Rev #6 Pepperoni & Sausage Pizza Wrap, ...
Name: name, dtype: object
